In [2]:
import os 
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader 
import pathlib
import glob

### mounting google drive on colab

In [3]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### extracting zip file

In [ ]:
import zipfile
import os

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/nature_12K.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/nature') #Extracts the files into the /nature folder
zip_ref.close()

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:


from torch.utils.data.sampler import SubsetRandomSampler

# how many samples per batch to load
batch_size = 64
# percentage of training set to use as validation
valid_size = 0.2

#To unzip dataset in colab
data_dir='/nature/inaturalist_12K'

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(200),
                                       transforms.RandomHorizontalFlip(),

                                       transforms.ToTensor(),
                                       
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(200),
                                     
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = torchvision.datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = torchvision.datasets.ImageFolder(data_dir + '/val', transform=test_transforms)
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
print(len(valid_loader))
